In [1]:
####################
#set up packages 
####################
import ee                     
ee.Initialize()
import pandas as pd
#import geemap.eefolium as emap
import csv
import time
from datetime import date
import numpy as np
import os, sys

In [4]:
start_time_total = time.time() #start runtime record

####################
#User defined functtions
####################
##create not overlapping zones along the hurricane track
def createBuffer(dist1,dist2,vector):
    buffer = vector.buffer(dist1).difference(vector.buffer(dist2),ee.ErrorMargin(1))
    return buffer  

##get patch vectors
def getImgVec(image,roi,threshold,resolution):
    vectors = image.clip(roi).gt(threshold)\
              .updateMask(image.clip(roi).gt(threshold))\
              .reduceToVectors(geometry= roi.geometry(),
                               scale= resolution,
                               geometryType= 'Polygon',
                               eightConnected=True,  #True or False?
                               maxPixels=1e12)
    return vectors

####################
#get hurricane tracks
####################
files = os.listdir('C:/Users/Y/Desktop/CHES/data/hurricane/ibtracsbyT/h5_since2001/')
#['Celia2010.csv',  'Dean2007.csv', 'Elida2002.csv', 'Emily2005.csv', 'Felix2007.csv', 'Hernan2002.csv', 'Ioke2006.csv',
#'Isabel2003.csv', 'Ivan2004.csv', 'Katrina2005.csv', 'Kenna2002.csv', 'Marie2014.csv', 'Matthew2016.csv', 'Patricia2015.csv',
#'Rick2009.csv','Rita2005.csv', 'Wilma2005.csv', 'Winston2016.csv']

#files = ['Rick2009.csv','Rita2005.csv', 'Wilma2005.csv', 'Winston2016.csv']

####################
#loop hurricane track
####################
for file in files:
    start_time_hurricane = time.time() 
    data = pd.read_csv('C:/Users/Y/Desktop/CHES/data/hurricane/ibtracsbyT/h5/'+ file) 
    Hyear = int(data.iloc[0]['Year'])
    Hname = data.iloc[0]['Name']
    data = data[['Longitude','Latitude']]
    coors = data.values.tolist()
    track = ee.Feature(ee.Geometry.LineString(coors),{'Name':Hname, 'Year': Hyear})
    print(track.getInfo()['properties'])

    trackBuffer = track.buffer(200000)    #create zones along the track
    buffer10k = track.buffer(10000)
    buffer20k = createBuffer(20000,10000,track)
    buffer30k = createBuffer(30000,20000,track)
    buffer40k = createBuffer(40000,30000,track)
    buffer50k = createBuffer(50000,40000,track)
    buffer60k = createBuffer(60000,50000,track)
    buffer70k = createBuffer(70000,60000,track)
    buffer80k = createBuffer(80000,70000,track)
    buffer90k = createBuffer(90000,80000,track)
    buffer100k = createBuffer(100000,90000,track)
    buffer110k = createBuffer(110000,100000,track)
    buffer120k = createBuffer(120000,110000,track)
    buffer130k = createBuffer(130000,120000,track)
    buffer140k = createBuffer(140000,130000,track)
    buffer150k = createBuffer(150000,140000,track)
    buffer160k = createBuffer(160000,150000,track)
    buffer170k = createBuffer(170000,160000,track)
    buffer180k = createBuffer(180000,170000,track)
    buffer190k = createBuffer(190000,180000,track)
    buffer200k = createBuffer(200000,190000,track)
    
####################
#load the vegetation map
####################
    mangrove_raw = ee.ImageCollection("LANDSAT/MANGROVE_FORESTS")\
                     .first()                            #2000
#    vege_rsl = mangrove_raw.projection().nominalScale().getInfo()   #get resolution    
    vege_rsl = 250
    mangrove_raw_vec = mangrove_raw.clip(trackBuffer.geometry()).eq(1)\
                                   .updateMask(mangrove_raw.clip(trackBuffer.geometry()).eq(1))\
                                   .reduceToVectors(geometry= trackBuffer.geometry(),
                                                    scale= vege_rsl,
                                                    geometryType= 'Polygon',
                                                    eightConnected=True,  #True or False?
                                                    maxPixels=1e13)      #convert image to vector
    
    if mangrove_raw_vec.size().getInfo() == 0:   #skip if not intersect with mangrove areas
        continue
    
    mangrove_raw_vec_lg = mangrove_raw_vec.filterMetadata('count','greater_than',16) #exclude patch smaller than 1km2, 10^6m2
    print('Vegetation pathches after filtering: ' + str(mangrove_raw_vec_lg.size().getInfo()))
    print('Total pixels after filtering: ' + str(mangrove_raw_vec_lg.aggregate_sum('count').getInfo()))
     
    if mangrove_raw_vec_lg.aggregate_max('count').getInfo() < 160:
        continue           #skip is the largest patch is smaller than 10km2
    
    mangrove = mangrove_raw_vec_lg.reduceToImage(['count'],'mean') #creat a map of vegetation with pathes size as pixel values
    mgvec10k = getImgVec(mangrove,buffer10k,1e6/vege_rsl/vege_rsl,vege_rsl) #vegetation zones
    mgvec20k = getImgVec(mangrove,buffer20k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec30k = getImgVec(mangrove,buffer30k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec40k = getImgVec(mangrove,buffer40k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec50k = getImgVec(mangrove,buffer50k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec60k = getImgVec(mangrove,buffer60k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec70k = getImgVec(mangrove,buffer70k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec80k = getImgVec(mangrove,buffer80k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec90k = getImgVec(mangrove,buffer90k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec100k = getImgVec(mangrove,buffer100k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec110k = getImgVec(mangrove,buffer110k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec120k = getImgVec(mangrove,buffer120k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec130k = getImgVec(mangrove,buffer130k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec140k = getImgVec(mangrove,buffer140k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec150k = getImgVec(mangrove,buffer150k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec160k = getImgVec(mangrove,buffer160k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec170k = getImgVec(mangrove,buffer170k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec180k = getImgVec(mangrove,buffer180k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec190k = getImgVec(mangrove,buffer190k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec200k = getImgVec(mangrove,buffer200k,1e6/vege_rsl/vege_rsl,vege_rsl)

####################
#vegetation condition one year before hurricane
####################
#    imgClt_before = ee.ImageCollection("MODIS/006/MOD13Q1")\
#                      .merge(ee.ImageCollection("MODIS/006/MYD13Q1"))\
#                      .filterDate(str(Hyear-1)+'-01-01', str(Hyear-1)+'-12-31')\
#                      .select(['EVI'])            
#    print('Before MODIS in '+ str(Hyear-1));
#    img_rsl = imgClt_before.first().projection().nominalScale().getInfo();
    img_rsl = 250
#    print('Satellite image resolution: ' + str(img_rsl))
#    imgClt_before_Vld = imgClt_before.map(lambda x: x.clip(trackBuffer).updateMask(x.gte(-2000).lte(10000)))  #only used pixels with valid values
#    mean = imgClt_before_Vld.reduce(ee.Reducer.mean())   #Get statistis
#    sd = imgClt_before_Vld.reduce(ee.Reducer.stdDev())
#    psd = sd.divide(mean)

    imgCltList = ["MODIS/006/MOD13Q1","MODIS/006/MYD13Q1"]
    vectorList = [mgvec10k, mgvec20k, mgvec30k, mgvec40k, mgvec50k, mgvec60k, mgvec70k, mgvec80k, mgvec90k, mgvec100k, 
                  mgvec110k, mgvec120k, mgvec130k, mgvec140k, mgvec150k, mgvec160k, mgvec170k, mgvec180k, mgvec190k, 
                  mgvec200k]
#    vectorList = [mgvec10k, mgvec20k, mgvec30k, mgvec40k, mgvec50k, mgvec60k, mgvec70k, mgvec80k, mgvec90k, mgvec100k]
    treshold = 3000 #set up treshold

####################
    for index in (['EVI', 'numberPatches','totalArea','maxPatch','meanPatch']): 
#index = 'EVI'
        print(index)
####################
        file = 'C:/Users/Y/Desktop/CHES/hurricane/result/downloaded/' + Hname + str(Hyear) + '_' + index + '_'\
                   + date.today().strftime('%Y-%m-%d') + '.csv'  #set up output file
        with open(file, 'w',newline='') as output:
            writer = csv.writer(output, delimiter=',')
            writer.writerow(['Collection'] + ['Index'] + ['Variable'] + ['Value'])   #write row names

        nameList =  [index + 'mg' + str(s) + 'k' for s in np.arange(10,210,10)] #create the df
        start_time = time.time()
                
        for v in range(0,len(vectorList)):
#        for v in range(0,1):
            vector = vectorList[v]
            name = nameList[v]
            print((v+1)*10, 'km')
    
#            file = 'C:/Users/Y/Desktop/CHES/hurricane/result/downloaded/' + Hname + str(Hyear) + '_' + name + '_'\
#                   + date.today().strftime('%Y-%m-%d') + '.csv'  #set up output file
#            with open(file, 'w',newline='') as output:
#                writer = csv.writer(output, delimiter=',')
#                writer.writerow(['Collection'] + ['Index'] + [name])
                
            for ic in range(0,len(imgCltList)):
#            for ic in range(0,1):
        
                for year in range(Hyear-2,Hyear+4):
#                for year in [2001]:
                    df = pd.DataFrame(columns=(['Collection'] + ['Index'] +['Variable'] + ['value'])) #create df 
                    imgClt = ee.ImageCollection(imgCltList[ic])\
                               .filterDate(str(year)+'-01-01', str(year)+'-12-31')\
                               .select(['EVI']) 
                    print(imgCltList[ic], 'in', year,'with', imgClt.size().getInfo() ,'images') 
                    if imgClt.size().getInfo() == 0:
                        continue
                    if index=='EVI':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index :x.clip(trackBuffer)\
 #                                               .updateMask(x.clip(trackBuffer).gte(-2000).lte(10000))\
 #                                               .updateMask(psd.lt(0.2))\
                                                .reduceRegion(geometry=vector,reducer=ee.Reducer.mean(),scale=img_rsl)\
                                                .get('EVI')}))
                
                    if index=='numberPatches':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index : getImgVec(x.clip(trackBuffer)\
#                                                           .updateMask(x.clip(trackBuffer).gte(-2000).lte(10000))\
#                                                           .updateMask(psd.lt(0.2))\
                                                ,vector,treshold,img_rsl).size()}))     
                    
                    if index=='totalArea':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index : getImgVec(x.clip(trackBuffer)\
#                                                           .updateMask(x.clip(trackBuffer).gte(-2000).lte(10000))\
#                                                           .updateMask(psd.lt(0.2))\
                                                ,vector,treshold,img_rsl).aggregate_sum('count')}))     
                    
                    if index=='maxPatch':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index : getImgVec(x.clip(trackBuffer)\
#                                                           .updateMask(x.clip(trackBuffer).gte(-2000).lte(10000))\
#                                                           .updateMask(psd.lt(0.2))\
                                                ,vector,treshold,img_rsl).aggregate_max('count')}))        
                   
                    if index=='meanPatch':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index : getImgVec(x.clip(trackBuffer)\
#                                                           .updateMask(x.clip(trackBuffer).gte(-2000).lte(10000))\
#                                                           .updateMask(psd.lt(0.2))\
                                                ,vector,treshold,img_rsl).aggregate_mean('count')}))     
                
                    listOfImages = imgClt_addedProperty.toList(imgClt_addedProperty.size()).getInfo()
                    indices = [x['properties']['system:index'] for x in listOfImages]
                    df['Index'] = indices
                    df['Collection'] = imgCltList[ic]
                    df['Variable'] = name
                    values = [0] * len(listOfImages)

                    for v in range(0,len(listOfImages)):
                        if index in listOfImages[v]['properties']: 
                            values[v] = listOfImages[v]['properties'][index]
                    df['value'] = values
          
                    with open(file, 'a',newline='') as output:
                        writer = csv.writer(output, delimiter=',')       #output data
                        for row in range(0,len(df.index)):
                            writer.writerow(df.loc[row,])
                  
        end_time = time.time()
        print('Runtime for', index, ':',(end_time-start_time)/60,'min')  # 11.154317851861318 min
    end_time_hurricane = time.time()
    print('Runtime for',Hname,':',(end_time_hurricane-start_time_hurricane)/60,'min')  #56.00555460453033 min    
end_time_total = time.time()
print('Total runtime:',(end_time_total-start_time_total)/60,'min') 

{'Name': 'CELIA', 'Year': 2010}
{'Name': 'DEAN', 'Year': 2007}
Vegetation pathches after filtering: 294
Total pixels after filtering: 54255
EVI
10 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
20 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 200

MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
180 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
190 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MO

MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
140 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
150 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MO

MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
100 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
110 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MO

MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
70 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODIS/006/MOD13Q1 in 2007 with 23 images
MODIS/006/MOD13Q1 in 2008 with 23 images
MODIS/006/MOD13Q1 in 2009 with 23 images
MODIS/006/MOD13Q1 in 2010 with 23 images
MODIS/006/MYD13Q1 in 2005 with 23 images
MODIS/006/MYD13Q1 in 2006 with 23 images
MODIS/006/MYD13Q1 in 2007 with 23 images
MODIS/006/MYD13Q1 in 2008 with 23 images
MODIS/006/MYD13Q1 in 2009 with 23 images
MODIS/006/MYD13Q1 in 2010 with 23 images
80 km
MODIS/006/MOD13Q1 in 2005 with 23 images
MODIS/006/MOD13Q1 in 2006 with 23 images
MODI

...

MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
80 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
90 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODI

MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
40 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
50 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODI

200 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
Runtime for totalArea : 4.611933843294779 min
maxPatch
10 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with

MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
170 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
180 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MO

MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
130 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MODIS/006/MYD13Q1 in 2016 with 23 images
MODIS/006/MYD13Q1 in 2017 with 23 images
MODIS/006/MYD13Q1 in 2018 with 23 images
MODIS/006/MYD13Q1 in 2019 with 23 images
140 km
MODIS/006/MOD13Q1 in 2014 with 23 images
MODIS/006/MOD13Q1 in 2015 with 23 images
MODIS/006/MOD13Q1 in 2016 with 23 images
MODIS/006/MOD13Q1 in 2017 with 23 images
MODIS/006/MOD13Q1 in 2018 with 23 images
MODIS/006/MOD13Q1 in 2019 with 23 images
MODIS/006/MYD13Q1 in 2014 with 23 images
MODIS/006/MYD13Q1 in 2015 with 23 images
MO

In [5]:
os.listdir('C:/Users/Y/Desktop/CHES/data/hurricane/ibtracsbyT/h5_since2001/')

['Celia2010_2010169N13266.csv',
 'Dean2007_2007225N12331.csv',
 'Elida2002_2002204N12263.csv',
 'Emily2005_2005192N11318.csv',
 'Felix2007_2007244N12303.csv',
 'Hernan2002_2002242N14257.csv',
 'Ioke2006_2006228N10218.csv',
 'Isabel2003_2003249N14329.csv',
 'Ivan2004_2004247N10332.csv',
 'Katrina2005_2005236N23285.csv',
 'Kenna2002_2002295N11261.csv',
 'Marie2014_2014234N12261.csv',
 'Matthew2016_2016273N13300.csv',
 'Patricia2015_2015293N13266.csv',
 'Rick2009_2009288N07267.csv',
 'Rita2005_2005261N21290.csv',
 'Wilma2005_2005289N18282.csv',
 'Winston2016_2016041S14170.csv']

In [ ]:
df